In [1]:
import pandas as pd

In [2]:
CDP = pd.read_excel(r"C:\Users\andre\OneDrive\Escritorio\DIP\01_Reporte de Ejecución Presupuestal\03_Código_Limpieza_Tablero\Resultados\2023-07-31\Archivos_Modificados\1_CDP.xlsx") 
RP = pd.read_excel(r"C:\Users\andre\OneDrive\Escritorio\DIP\01_Reporte de Ejecución Presupuestal\03_Código_Limpieza_Tablero\Resultados\2023-07-31\Archivos_Modificados\2_RP.xlsx")
OB = pd.read_excel(r"C:\Users\andre\OneDrive\Escritorio\DIP\01_Reporte de Ejecución Presupuestal\03_Código_Limpieza_Tablero\Resultados\2023-07-31\Archivos_Modificados\3_Oblig.xlsx")
OP = pd.read_excel(r"C:\Users\andre\OneDrive\Escritorio\DIP\01_Reporte de Ejecución Presupuestal\03_Código_Limpieza_Tablero\Resultados\2023-07-31\Archivos_Modificados\4_OP.xlsx")

In [3]:
CDP = CDP[CDP['Dirección']=='Inclusión Productiva'].reset_index(drop=True)
RP = RP[RP['Dirección']=='Inclusión Productiva'].reset_index(drop=True)
OB = OB[OB['Dirección']=='Inclusión Productiva'].reset_index(drop=True)
OP = OP[OP['Dirección']=='Inclusión Productiva'].reset_index(drop=True)

In [4]:
CDP['Rubro'] = CDP['Rubro'].apply(lambda x: x[:14])
RP['Rubro'] = RP['Rubro'].apply(lambda x: x[:14])
OB['Rubro'] = OB['Rubro'].apply(lambda x: x[:14])
OP['Rubro'] = OP['Rubro'].apply(lambda x: x[:14])

In [5]:
OP = OP[OP['Estado']=='Pagada'].reset_index(drop = True)

In [6]:
RP = RP[['Numero Documento Soporte','Descripción Rubro','Rubro','Valor Actual','CDP','Compromisos','Tipo Identificacion','Tipo Documento Soporte','Nombre Razon Social','Descripcion',]]

In [7]:
gr1 = RP.groupby(by='Numero Documento Soporte').agg({'Descripción Rubro':lambda x: len(pd.unique(x))}).reset_index(drop=False)

In [8]:
Honorarios = [9523,24623,9723,9823]
Viaticos = [10823,25523, 11023,10923]
Contrato = [26723]

In [9]:
RP.loc[RP['CDP'] == 26823, 'Numero Documento Soporte'] = RP.loc[RP['CDP'] == 26823, 'Numero Documento Soporte'].str.replace("FIP", "DPS")

In [10]:
RP['OPS'] = RP['CDP'].apply(lambda x: 1 if x in Honorarios else 0)
RP['Viaticos'] = RP['CDP'].apply(lambda x: 1 if x in Viaticos else 0)

In [11]:
RP['Transversal'] = 0
for i in range(len(RP)):
    if RP['OPS'][i] == 1 or RP['Viaticos'][i] == 1:
        RP.loc[i, 'Transversal'] = 1
    if gr1[gr1['Numero Documento Soporte'] == RP['Numero Documento Soporte'][i]]['Descripción Rubro'].iat[0]>1:
        RP.loc[i, 'Transversal'] = 1

In [12]:
gr1[gr1['Numero Documento Soporte']==RP['Numero Documento Soporte'][79]]['Descripción Rubro'].iat[0]

3

In [13]:
CDP['Rubro'] = CDP.apply(lambda x: ' '.join([str(x['Descripción Rubro']), str(x['Rubro'])]), axis = 1)
RP['Rubro'] = RP.apply(lambda x: ' '.join([str(x['Descripción Rubro']), str(x['Rubro'])]), axis = 1)
OB['Rubro'] = OB.apply(lambda x: ' '.join([str(x['Descripción Rubro']), str(x['Rubro'])]), axis = 1)
OP['Rubro'] = OP.apply(lambda x: ' '.join([str(x['Descripción Rubro']), str(x['Rubro'])]), axis = 1)

In [14]:
OPS = RP[RP['OPS']==1].reset_index(drop=True)
Via = RP[RP['Viaticos']==1].reset_index(drop=True)
Trans = RP[(RP['Transversal']==1) & (RP['OPS']==0) & (RP['Viaticos']==0)].reset_index(drop=True)
No_Trans = RP[RP['Transversal']!=1]

In [15]:
OPS = OPS.groupby(by="Rubro").agg({'Valor Actual':'sum',
                                   'CDP':'first',
                                   'Compromisos':'unique'}).reset_index(drop=False)
OPS['Tipo Gasto'] = 'Valor de OPS'
OPS['Tipo Gasto_2'] = 'T1'

In [16]:
Via = Via.groupby(by="Rubro").agg({'Valor Actual':'sum',
                                   'CDP':'first',
                                   'Compromisos':'unique'}).reset_index(drop=False)
Via['Tipo Gasto'] = 'Viáticos y Gastos de Viaje Contratistas'
Via['Tipo Gasto_2'] = 'T2'

In [17]:
No_Trans['Tipo Gasto'] = No_Trans.apply(lambda x: ' '.join([str(x['Tipo Documento Soporte']),str(x['Numero Documento Soporte']), str(x['Nombre Razon Social'])]), axis=1)
No_Trans = No_Trans.groupby(by=['Rubro','Tipo Gasto']).agg({'Valor Actual':'sum',
                                                               'CDP':'first',
                                   'Compromisos':'unique'}).reset_index(drop=False)
No_Trans['Tipo Gasto_2'] = 'A'

C:\Users\andre\AppData\Local\Temp\ipykernel_6720\2591207830.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  No_Trans['Tipo Gasto'] = No_Trans.apply(lambda x: ' '.join([str(x['Tipo Documento Soporte']),str(x['Numero Documento Soporte']), str(x['Nombre Razon Social'])]), axis=1)


In [18]:
Trans['Tipo Gasto'] = Trans.apply(lambda x: ' '.join([str(x['Tipo Documento Soporte']),str(x['Numero Documento Soporte']), str(x['Nombre Razon Social'])]), axis=1)
Trans = Trans.groupby(by=['Rubro','Tipo Gasto']).agg({'Valor Actual':'sum',
                                                               'CDP':'first',
                                   'Compromisos':'unique'}).reset_index(drop=False)
Trans['Tipo Gasto_2'] = 'T3'

In [19]:
result = pd.concat([No_Trans,OPS,Via,Trans], axis=0).sort_values(by=['Rubro','Tipo Gasto_2']).reset_index(drop=True)
result = result.rename(columns={'Valor Actual':'Vr. COMPROMISO'})

In [20]:
CDP.head()

,Numero Documento,Fecha de Registro,Fecha de Creacion,Tipo de CDP,Estado,Dependencia,Dependencia Descripcion,Rubro,Descripcion,Fuente,...,Compromisos,Cuentas por Pagar,Obligaciones,Ordenes de Pago,Reintegros,Index_Rubro,Descripción Rubro,Dirección,No. Mes,Mes
0,7523,2023-01-04 00:00:00,2023-01-04 16:51:00,Gasto,Con Compromiso,0,PS GESTION GENERAL,FEST C-4103-1500-22,ADQUISICIÓN DE BIENES Y SERVICIOS - SERVICIO D...,Nación,...,6223,13823,"64223, 125323, 238123","36971523, 74644423, 140423123",-0,22,FEST,Inclusión Productiva,1,Enero
1,8323,2023-01-05 00:00:00,2023-01-05 11:10:00,Gasto,Con Compromiso,0,PS GESTION GENERAL,ReSA C-4103-1500-13,ADQUISICIÓN DE BIENES Y SERVICIOS - SERVICIO D...,Nación,...,7823,-0,-0,-0,-0,13,ReSA,Inclusión Productiva,1,Enero
2,8323,2023-01-05 00:00:00,2023-01-05 11:10:00,Gasto,Con Compromiso,0,PS GESTION GENERAL,ReSA C-4103-1500-13,ADQUISICIÓN DE BIENES Y SERVICIOS - SERVICIO D...,Nación,...,7823,-0,-0,-0,-0,13,ReSA,Inclusión Productiva,1,Enero
3,8323,2023-01-05 00:00:00,2023-01-05 11:10:00,Gasto,Con Compromiso,0,PS GESTION GENERAL,ReSA C-4103-1500-13,ADQUISICIÓN DE BIENES Y SERVICIOS - SERVICIO D...,Nación,...,7823,-0,-0,-0,-0,13,ReSA,Inclusión Productiva,1,Enero
4,8423,2023-01-05 00:00:00,2023-01-05 11:34:00,Gasto,Con Compromiso,0,PS GESTION GENERAL,FEST C-4103-1500-22,ADQUISICIÓN DE BIENES Y SERVICIOS - SERVICIO D...,Nación,...,7923,-0,-0,-0,-0,22,FEST,Inclusión Productiva,1,Enero


In [21]:
result['Compromisos'] = result['Compromisos'].apply(lambda x: x.tolist())
CDP['Compromisos'] = CDP['Compromisos'].str.split(",").tolist()
CDP['Compromisos'] = CDP['Compromisos'].apply(lambda x: [int(i) for i in (x)])

In [22]:
filtered_df2 = CDP[CDP['Compromisos'].apply(lambda x: any(val in x for val in result['Compromisos'][1]))]

In [23]:
def get_valor_CDP(data_cdp, base, index):
    filter_1 = data_cdp[(data_cdp['Numero Documento'] == base['CDP'][index]) & (data_cdp['Rubro'] == base['Rubro'][index])]
    filter_2 = filter_1[filter_1['Compromisos'].apply(lambda x: any(val in x for val in base['Compromisos'][index]))]
    value = filter_2['Valor Actual'].sum()
    return value

In [24]:
get_valor_CDP(CDP,result,1)

8721058504.0

In [25]:
result['Vr. CDP'] = 0

In [26]:
for i in range(len(result)):
    result.loc[i, 'Vr. CDP'] = get_valor_CDP(CDP, result, i)

In [27]:
result

,Rubro,Tipo Gasto,Vr. COMPROMISO,CDP,Compromisos,Tipo Gasto_2,Vr. CDP
0,FEST C-4103-1500-22,CONTRATO INTERADMINISTRATIVO 222 DPS 2023 OTRO...,3.000000e+08,13023,[86423],A,300000000
1,FEST C-4103-1500-22,CONVENIO 440 DPS 2022 OTROSI 1 CONSORCIO FEST ...,8.721059e+09,8423,[7923],A,8721058504
2,FEST C-4103-1500-22,ORDEN ADMINISTRATIVA M-2022-4200-067324/ 03378...,0.000000e+00,8523,[8023],A,33298512000
3,FEST C-4103-1500-22,Valor de OPS,2.175327e+09,9823,"[22823, 22923, 28223, 32523, 38123, 38423, 385...",T1,3052800000
4,FEST C-4103-1500-22,Viáticos y Gastos de Viaje Contratistas,1.779725e+07,10923,"[70623, 92223, 107423, 119423, 199523, 221623,...",T2,90000000
5,FEST C-4103-1500-22,CONTRATO INTERADMINISTRATIVO 341 DPS 2023 OTRO...,1.037030e+08,14023,[98123],T3,103702985
6,FEST C-4103-1500-22,ORDEN DE COMPRA 108977 DPS 2023 CONTROLES EMPR...,2.000000e+08,11423,[163023],T3,200000000
7,IRACA C-4103-1500-21,Valor de OPS,1.059217e+09,9723,"[32723, 38623, 41823, 45823, 48523, 55323, 611...",T1,1577200000
8,IRACA C-4103-1500-21,Viáticos y Gastos de Viaje Contratistas,2.790939e+06,11023,"[73823, 85523, 110623, 128523, 144423, 163523,...",T2,60000000
9,IRACA C-4103-1500-21,CONTRATO DE PRESTACION DE SERVICIOS - PROFESIO...,2.354552e+07,11223,[225823],T3,75777342


In [28]:
result['Vr. OBLIGADO'] = result['Compromisos'].apply(lambda x: OB[OB['Compromisos'].isin(x)]['Valor Actual.1'].sum())
result['Vr. PAGADO'] = result['Compromisos'].apply(lambda x: OP[OP['Compromisos'].isin(x)]['Valor Neto Pesos'].sum())

In [29]:
muestra = result[['Tipo Gasto_2','Rubro','Tipo Gasto','Vr. CDP','Vr. COMPROMISO','Vr. OBLIGADO','Vr. PAGADO']]

In [30]:
RP['Descripcion'][1]

'ADQUISICIÓN DE BIENES Y SERVICIOS - SERVICIO DE ASISTENCIA TÉCNICA PARA EL MEJORAMIENTO DE HÁBITOS ALIMENTARIOS - IMPLEMENTACIÓN DE UNIDADES PRODUCTIVAS DE AUTOCONSUMO PARA POBLACIÓN POBRE Y VULNERABLE NACIONAL'

In [31]:
CDP[~CDP['Numero Documento'].isin(result['CDP'].to_list())]

,Numero Documento,Fecha de Registro,Fecha de Creacion,Tipo de CDP,Estado,Dependencia,Dependencia Descripcion,Rubro,Descripcion,Fuente,...,Compromisos,Cuentas por Pagar,Obligaciones,Ordenes de Pago,Reintegros,Index_Rubro,Descripción Rubro,Dirección,No. Mes,Mes
0,7523,2023-01-04 00:00:00,2023-01-04 16:51:00,Gasto,Con Compromiso,0,PS GESTION GENERAL,FEST C-4103-1500-22,ADQUISICIÓN DE BIENES Y SERVICIOS - SERVICIO D...,Nación,...,[6223],13823,"64223, 125323, 238123","36971523, 74644423, 140423123",-0,22,FEST,Inclusión Productiva,1,Enero
30,10023,2023-01-12 00:00:00,2023-01-12 09:54:00,Gasto,Generado,0,PS GESTION GENERAL,FEST C-4103-1500-22,ADQUISICIÓN DE BIENES Y SERVICIOS - SERVICIO D...,Nación,...,[0],-0,-0,-0,-0,22,FEST,Inclusión Productiva,1,Enero
37,11323,2023-02-07 00:00:00,2023-02-07 10:56:00,Gasto,Con Compromiso,0,PS GESTION GENERAL,FEST C-4103-1500-22,ADQUISICIÓN DE BIENES Y SERVICIOS - SERVICIO D...,Nación,...,"[224723, 226023]",-0,-0,-0,-0,22,FEST,Inclusión Productiva,2,Febrero
38,11323,2023-02-07 00:00:00,2023-02-07 10:56:00,Gasto,Con Compromiso,0,PS GESTION GENERAL,IRACA C-4103-1500-21,ADQUISICION DE BIENES Y SERVICIOS - SERVICIO ...,Nación,...,"[224723, 226023]",-0,-0,-0,-0,21,IRACA,Inclusión Productiva,2,Febrero
39,11323,2023-02-07 00:00:00,2023-02-07 10:56:00,Gasto,Con Compromiso,0,PS GESTION GENERAL,ReSA C-4103-1500-13,ADQUISICIÓN DE BIENES Y SERVICIOS - SERVICIO D...,Nación,...,"[224723, 226023]",-0,-0,-0,-0,13,ReSA,Inclusión Productiva,2,Febrero
43,11623,2023-02-14 00:00:00,2023-02-14 08:51:00,Gasto,Generado,0,PS GESTION GENERAL,ReSA C-4103-1500-13,ADQUISICIÓN DE BIENES Y SERVICIOS - SERVICIO D...,Nación,...,[0],-0,-0,-0,-0,13,ReSA,Inclusión Productiva,2,Febrero
49,15423,2023-04-21 00:00:00,2023-04-21 09:48:00,Gasto,Generado,0,PS GESTION GENERAL,IRACA C-4103-1500-21,ADQUISICION DE BIENES Y SERVICIOS - SERVICIO ...,Nación,...,[0],-0,-0,-0,-0,21,IRACA,Inclusión Productiva,4,Abril
50,15423,2023-04-21 00:00:00,2023-04-21 09:48:00,Gasto,Generado,0,PS GESTION GENERAL,IRACA C-4103-1500-21,ADQUISICION DE BIENES Y SERVICIOS - SERVICIO ...,Nación,...,[0],-0,-0,-0,-0,21,IRACA,Inclusión Productiva,4,Abril
51,15423,2023-04-21 00:00:00,2023-04-21 09:48:00,Gasto,Generado,0,PS GESTION GENERAL,IRACA C-4103-1500-21,ADQUISICION DE BIENES Y SERVICIOS - SERVICIO ...,Nación,...,[0],-0,-0,-0,-0,21,IRACA,Inclusión Productiva,4,Abril
52,15423,2023-04-21 00:00:00,2023-04-21 09:48:00,Gasto,Generado,0,PS GESTION GENERAL,IRACA C-4103-1500-21,ADQUISICION DE BIENES Y SERVICIOS - SERVICIO ...,Nación,...,[0],-0,-0,-0,-0,21,IRACA,Inclusión Productiva,4,Abril


In [32]:
result

,Rubro,Tipo Gasto,Vr. COMPROMISO,CDP,Compromisos,Tipo Gasto_2,Vr. CDP,Vr. OBLIGADO,Vr. PAGADO
0,FEST C-4103-1500-22,CONTRATO INTERADMINISTRATIVO 222 DPS 2023 OTRO...,3.000000e+08,13023,[86423],A,300000000,3.000000e+08,300000000.0
1,FEST C-4103-1500-22,CONVENIO 440 DPS 2022 OTROSI 1 CONSORCIO FEST ...,8.721059e+09,8423,[7923],A,8721058504,0.000000e+00,0.0
2,FEST C-4103-1500-22,ORDEN ADMINISTRATIVA M-2022-4200-067324/ 03378...,0.000000e+00,8523,[8023],A,33298512000,0.000000e+00,0.0
3,FEST C-4103-1500-22,Valor de OPS,2.175327e+09,9823,"[22823, 22923, 28223, 32523, 38123, 38423, 385...",T1,3052800000,9.134100e+08,833116664.0
4,FEST C-4103-1500-22,Viáticos y Gastos de Viaje Contratistas,1.779725e+07,10923,"[70623, 92223, 107423, 119423, 199523, 221623,...",T2,90000000,4.648604e+06,4648604.0
5,FEST C-4103-1500-22,CONTRATO INTERADMINISTRATIVO 341 DPS 2023 OTRO...,1.037030e+08,14023,[98123],T3,103702985,2.406330e+08,240633005.0
6,FEST C-4103-1500-22,ORDEN DE COMPRA 108977 DPS 2023 CONTROLES EMPR...,2.000000e+08,11423,[163023],T3,200000000,4.000000e+08,400000000.0
7,IRACA C-4103-1500-21,Valor de OPS,1.059217e+09,9723,"[32723, 38623, 41823, 45823, 48523, 55323, 611...",T1,1577200000,3.847767e+08,384776665.0
8,IRACA C-4103-1500-21,Viáticos y Gastos de Viaje Contratistas,2.790939e+06,11023,"[73823, 85523, 110623, 128523, 144423, 163523,...",T2,60000000,1.905824e+06,1905824.0
9,IRACA C-4103-1500-21,CONTRATO DE PRESTACION DE SERVICIOS - PROFESIO...,2.354552e+07,11223,[225823],T3,75777342,0.000000e+00,0.0


In [33]:
PAABS = pd.read_excel(r"C:\Users\andre\OneDrive\Escritorio\DIP\07_Tablero_PAA\Reporte_PAABS(20).xlsx", header = 5)

In [34]:
PAABS.columns

Index(['Unnamed: 0', 'DEPENDENCIA', 'Unnamed: 2', 'CODIGO_PAABS', 'Unnamed: 4',
       'OBJETO', 'ESTADO_PAABS', 'AÑO', 'RUBRO', 'VALOR_ASIGNADO_RUBRO',
       'RECURSO', 'SUB_UNIDAD', 'MES_ESTIMADO', 'PLAZO_ESTIMADO',
       'PRESUPUESTO_ESTIMADO', 'ACTIVO', 'TIPO_CONTRATACION',
       'REQUIERE_CONTRATACION', 'REQUIERE_INV_MERCADO',
       'VALOR_VIGENCIA_FUTURA', 'TOTAL CDPS'],
      dtype='object')

In [35]:
PAABS = PAABS.drop(columns=['Unnamed: 0', 'Unnamed: 2', 'Unnamed: 4'])

In [36]:
PAABS.head()

,DEPENDENCIA,CODIGO_PAABS,OBJETO,ESTADO_PAABS,AÑO,RUBRO,VALOR_ASIGNADO_RUBRO,RECURSO,SUB_UNIDAD,MES_ESTIMADO,PLAZO_ESTIMADO,PRESUPUESTO_ESTIMADO,ACTIVO,TIPO_CONTRATACION,REQUIERE_CONTRATACION,REQUIERE_INV_MERCADO,VALOR_VIGENCIA_FUTURA,TOTAL CDPS
0,DIRECCIÓN DE INCLUSIÓN PRODUCTIVA,1209723,COMISIONES BANCARIAS INCENTIVOS MONETARIOS FES...,Ordenado,2023,C-4103-1500-22-0-4103062-02 - IMPLEMENTACIÓN D...,26545007670,11 - OTROS RECURSOS DEL TESORO,000 - DPS,7,6.0,110858384,SI,Selección abreviada – Acuerdo marco de precios,SI,NO,0,110858384
1,DIRECCIÓN DE INCLUSIÓN PRODUCTIVA,1209623,PAGO INCENTIVOS MONETARIOS CONDICIONADOS FEST ...,Ordenado,2023,C-4103-1500-22-0-4103062-03 - IMPLEMENTACIÓN D...,12906400000,11 - OTROS RECURSOS DEL TESORO,000 - DPS,-1,-1.0,12906400000,SI,NaN,NO,NO,0,12906400000
2,DIRECCIÓN DE INCLUSIÓN PRODUCTIVA,1209523,CONTRATAR UN OPERADOR PARA LA ATENCIÓN FEST ÉT...,Ordenado,2023,C-4103-1500-22-0-4103062-02 - IMPLEMENTACIÓN D...,26545007670,11 - OTROS RECURSOS DEL TESORO,000 - DPS,7,6.0,766229074,SI,Contratación Directa,SI,NO,0,766229074
3,DIRECCIÓN DE INCLUSIÓN PRODUCTIVA,1209423,COMISIONES BANCARIAS INCENTIVOS MONETARIOS FES...,Ordenado,2023,C-4103-1500-22-0-4103062-02 - IMPLEMENTACIÓN D...,26545007670,11 - OTROS RECURSOS DEL TESORO,000 - DPS,7,6.0,70659078,SI,Selección abreviada – Acuerdo marco de precios,SI,NO,0,70659078
4,DIRECCIÓN DE INCLUSIÓN PRODUCTIVA,1209323,CONTRATAR UN OPERADOR PARA LA ATENCIÓN FEST ÉT...,Ordenado,2023,C-4103-1500-22-0-4103057-02 - IMPLEMENTACIÓN D...,6826563656,11 - OTROS RECURSOS DEL TESORO,000 - DPS,7,6.0,134076088,SI,Contratación Directa,SI,NO,0,134076088


In [37]:
PAABS['RUBRO'][1][:14]

'C-4103-1500-22'

In [38]:
PAABS['Código Rubro'] = PAABS['RUBRO'].apply(lambda x: str(x)[:14])

In [39]:
PAABS['Código Rubro'].value_counts()

Código Rubro
C-4103-1500-22    44
C-4103-1500-13    33
C-4103-1500-21    15
C-4103-1500-17    12
C-4103-1500-25     4
C-4103-1500-16     2
Name: count, dtype: int64

In [40]:
DPAABS = PAABS.drop_duplicates(subset=['PRESUPUESTO_ESTIMADO','OBJETO','CODIGO_PAABS'], keep='first')

In [41]:
DPAABS.info()

<class 'pandas.core.frame.DataFrame'>
Index: 94 entries, 0 to 104
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DEPENDENCIA            94 non-null     object 
 1   CODIGO_PAABS           94 non-null     int64  
 2   OBJETO                 94 non-null     object 
 3   ESTADO_PAABS           94 non-null     object 
 4   AÑO                    94 non-null     int64  
 5   RUBRO                  94 non-null     object 
 6   VALOR_ASIGNADO_RUBRO   94 non-null     int64  
 7   RECURSO                94 non-null     object 
 8   SUB_UNIDAD             94 non-null     object 
 9   MES_ESTIMADO           94 non-null     int64  
 10  PLAZO_ESTIMADO         94 non-null     float64
 11  PRESUPUESTO_ESTIMADO   94 non-null     int64  
 12  ACTIVO                 94 non-null     object 
 13  TIPO_CONTRATACION      77 non-null     object 
 14  REQUIERE_CONTRATACION  94 non-null     object 
 15  REQUIERE_INV

In [42]:
DPAABS = DPAABS.iloc[:,[1,2,3,5,9,11,12,17,18]]

In [43]:
print(PAABS.shape)
print(DPAABS.shape)

(110, 19)
(94, 9)


In [44]:
DPAABS.groupby(by='Código Rubro').sum()['PRESUPUESTO_ESTIMADO']

Código Rubro
C-4103-1500-13     40000000000
C-4103-1500-16       500000000
C-4103-1500-17     18000000000
C-4103-1500-21     34000000000
C-4103-1500-22    188182165239
C-4103-1500-25      5500000000
Name: PRESUPUESTO_ESTIMADO, dtype: int64

In [45]:
DPAABS['OBJETO'][12]

'Adquirir equipos de cómputo y de apoyo informático'

In [46]:
CDP['Objeto'][12]

'IMPLEMENTAR UNIDADES PRODUCTIVAS DE AUTOCONSUMO PARA LOS HOGARES DE LAS COMUNIDADES ÉTNICAS, TRAVÉS DEL PROGRAMA RESA®, ARTICULANDO ESFUERZOS'

In [47]:
CDP.columns

Index(['Numero Documento', 'Fecha de Registro', 'Fecha de Creacion',
       'Tipo de CDP', 'Estado', 'Dependencia', 'Dependencia Descripcion',
       'Rubro', 'Descripcion', 'Fuente', 'Recurso', 'Sit', 'Valor Inicial',
       'Valor Operaciones', 'Valor Actual', 'Saldo por Comprometer', 'Objeto',
       'Solicitud CDP', 'Compromisos', 'Cuentas por Pagar', 'Obligaciones',
       'Ordenes de Pago', 'Reintegros', 'Index_Rubro', 'Descripción Rubro',
       'Dirección', 'No. Mes', 'Mes'],
      dtype='object')

In [48]:
CP1 = CDP.groupby(by=['Rubro','Objeto']).agg({'Valor Actual':'sum','Numero Documento':'first'}).reset_index()

In [49]:
PA1 = DPAABS.groupby(by=['OBJETO','Código Rubro']).agg({'PRESUPUESTO_ESTIMADO':'sum'}).reset_index()

In [50]:
PA1.head()

,OBJETO,Código Rubro,PRESUPUESTO_ESTIMADO
0,Adquirir la extensión de los servicios de nub...,C-4103-1500-21,100000000
1,CONTRATAR OPERADORES PARA LA ATENCIÓN HOGARE...,C-4103-1500-22,11496229210
2,CONTRATAR OPERADORES PARA LA ATENCIÓN HOGARE...,C-4103-1500-22,4247205166
3,Contratar un operador / socio implementador p...,C-4103-1500-13,1801727058
4,PAGO INCENTIVOS MONETARIOS CONDICIONADOS FEST IX,C-4103-1500-22,20458371400


In [51]:
PA1.shape

(70, 3)

In [52]:
CP1['RUBRO_2'] = CP1['Rubro'].apply(lambda x: x.split(" ")[1])
CP1.head()

,Rubro,Objeto,Valor Actual,Numero Documento,RUBRO_2
0,FEST C-4103-1500-22,ADQUIRIR LA EXTENSON DE LOS SERVICIOS DE NUBE ...,2.000000e+08,11423,C-4103-1500-22
1,FEST C-4103-1500-22,ADQUIRIR LA RENOVACION DE LICENCIAMIENTO MICRO...,4.567000e+07,11323,C-4103-1500-22
2,FEST C-4103-1500-22,"AUNAR ESFUERZOS TÉCNICOS, ADMINISTRATIVOS Y FI...",8.721059e+09,8423,C-4103-1500-22
3,FEST C-4103-1500-22,COMISIONES BANCARIAS INCENTIVOS MONETARIOS FES...,1.474056e+08,16423,C-4103-1500-22
4,FEST C-4103-1500-22,"CONTRATAR EL MANTENIMIENTO, SOPORTE, ACTUALIZA...",7.577734e+07,11223,C-4103-1500-22


In [53]:
CP1[(CP1['Valor Actual'] == PA1['PRESUPUESTO_ESTIMADO'][8]) & (CP1['RUBRO_2'] == PA1['Código Rubro'][8])]

,Rubro,Objeto,Valor Actual,Numero Documento,RUBRO_2


In [64]:
PA1[PA1['OBJETO'].str.contains(" CONTRATAR  OPERADORES")].sum()

OBJETO                   CONTRATAR  OPERADORES PARA LA ATENCIÓN HOGARE...
Código Rubro                                 C-4103-1500-22C-4103-1500-22
PRESUPUESTO_ESTIMADO                                          15743434376
dtype: object